In [59]:
import numpy as np
import pandas as pd
import pickle
import time

In [60]:
myfile = open('pickles/DI.pkl', 'rb')
X = pickle.load(myfile)
myfile.close()
print len(X), X.iloc[0].shape

myfile = open('pickles/used_column_headers.pkl', 'rb')
column_headers = pickle.load(myfile)
myfile.close()
print len(column_headers)

myfile = open('pickles/ydf.pkl', 'rb')
y = pickle.load(myfile)
myfile.close()
print y.head()

1078 (60, 220)
220
           classification gram  detection
0        Escherichia_coli    n          1
1        Escherichia_coli    n          1
2  Pseudomonas_aeruginosa    n          1
3  Pseudomonas_aeruginosa    n          1
4  Pseudomonas_aeruginosa    n          1


In [61]:
Ncolors = 3
Nsamp = len(X)
Ntimes = 60

col_nums = list(set([int(x[:-1]) for x in column_headers[1:]]))
Nspots = len(col_nums)

tidxs = y.index.values

# Convert series to frame
Z = X.to_frame()

# flatten arrays in data frame
start = time.time()
print 'Flattening'
final = np.array(Z['data'].iloc[0])
for i in xrange(1, Nsamp):
    final = np.vstack((final, Z['data'].iloc[i]))
    
end = time.time()
print 'Flattened %d trials in %d secs' % (Nsamp, (end-start))

Flattening
Flattened 1078 trials in 57 secs


In [62]:
print final.shape, Nsamp*Ntimes, 1+Ncolors*Nspots, Nsamp*Ntimes*Nspots, Nsamp*Ntimes*Ncolors, Nsamp*Ntimes*Nspots*Ncolors

(64680, 220) 64680 220 4721640 194040 14164920


In [63]:
data = final[:,1:]

In [64]:
data = data.reshape((Ntimes*Nsamp*Nspots,3))
print data.shape, Ntimes*Nsamp, Ntimes*Nsamp*Nspots

(4721640, 3) 64680 4721640


In [65]:
df = pd.DataFrame(data=data, columns=['R', 'G', 'B'])
df.head()

,R,G,B
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [66]:
df['spot'] = np.array([x for x in col_nums]*(Nsamp*Ntimes)).flatten()
df.head()

,R,G,B,spot
0,0,0,0,2
1,0,0,0,3
2,0,0,0,4
3,0,0,0,5
4,0,0,0,6


In [67]:
df['trial'] = np.array([[x]*(Ntimes*Nspots) for x in tidxs]).flatten()
df['detection'] = np.array([[y.ix[tidxs[x]]['detection']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df['gram'] = np.array([[y.ix[tidxs[x]]['gram']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df['classification'] = np.array([[y.ix[tidxs[x]]['classification']]*(Ntimes*Nspots) for x in range(Nsamp)]).flatten()
df.head()

,R,G,B,spot,trial,detection,gram,classification
0,0,0,0,2,0,1,n,Escherichia_coli
1,0,0,0,3,0,1,n,Escherichia_coli
2,0,0,0,4,0,1,n,Escherichia_coli
3,0,0,0,5,0,1,n,Escherichia_coli
4,0,0,0,6,0,1,n,Escherichia_coli


In [68]:
df['time'] = np.array([[[20*x]*Nspots for x in range(Ntimes)]*Nsamp]).flatten()
df.head()
df.iloc[72:75]

,R,G,B,spot,trial,detection,gram,classification,time
72,0,0,0,79,0,1,n,Escherichia_coli,0
73,0,0,0,2,0,1,n,Escherichia_coli,20
74,0,0,0,3,0,1,n,Escherichia_coli,20


In [69]:
# save to R
from rpy2.robjects import pandas2ri, r
import pickle
pandas2ri.activate()
rX = pandas2ri.py2ri(df)
r.assign('RGB', rX)
r("save(RGB, file='RGB.gzip', compress=TRUE)")
pandas2ri.deactivate()

In [79]:
len(df), Nsamp*Ntimes*Nspots, len(newdf), Nsamp*Ntimes

(4721640, 4721640, 64680, 64680)

In [70]:
# Experiment with time re-scaling
newdf = pd.DataFrame(final)

In [71]:
newdf.columns = column_headers
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64680 entries, 0 to 64679
Columns: 220 entries, time to 79B
dtypes: float64(220)
memory usage: 109.1 MB


In [72]:
print len(newdf), Nsamp*Ntimes, len(tidxs)

64680 64680 1078


In [73]:
# trial index
newdf['trial'] = np.array([[x]*(Ntimes) for x in tidxs]).flatten()
newdf['detection'] = np.array([[y.ix[tidxs[x]]['detection']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf['gram'] = np.array([[y.ix[tidxs[x]]['gram']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf['classification'] = np.array([[y.ix[tidxs[x]]['classification']]*(Ntimes) for x in range(Nsamp)]).flatten()
newdf.head()

,time,2R,2G,2B,3R,3G,3B,4R,4G,4B,...,78R,78G,78B,79R,79G,79B,trial,detection,gram,classification
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,n,Escherichia_coli
1,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,n,Escherichia_coli
2,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,n,Escherichia_coli
3,60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,n,Escherichia_coli
4,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,n,Escherichia_coli


In [112]:
avgdf = newdf.groupby(['classification', 'time']).mean()
print len(avgdf), len(avgdf)/Ntimes
avgdf.head()
avgdf = avgdf.reset_index()
avgdf.head()

1080 18


,classification,time,2R,2G,2B,3R,3G,3B,4R,4G,...,77G,77B,78R,78G,78B,79R,79G,79B,trial,detection
0,Acinetobacter_baumannii,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,689.76,1
1,Acinetobacter_baumannii,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,689.76,1
2,Acinetobacter_baumannii,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,689.76,1
3,Acinetobacter_baumannii,60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,689.76,1
4,Acinetobacter_baumannii,80,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,689.76,1


In [116]:
def create_labels_dictionaries(species_info_filename = 'species_to_gram2.csv'):
    label_df = pd.read_csv(species_info_filename, index_col=0)

    # label_dictionaries = {{column_name: label_df[column_name].to_dict()} for column_name in label_df.columns}
    label_dictionaries = {}
    for column_name in label_df.columns:
        label_dictionaries[column_name] = label_df[column_name].to_dict()

    return label_dictionaries
ld = create_labels_dictionaries()
avgdf['gram'] = avgdf['classification'].apply(lambda x: ld['gram'][x])
avgdf.head()

,classification,time,2R,2G,2B,3R,3G,3B,4R,4G,...,77B,78R,78G,78B,79R,79G,79B,trial,detection,gram
0,Acinetobacter_baumannii,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,689.76,1,n
1,Acinetobacter_baumannii,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,689.76,1,n
2,Acinetobacter_baumannii,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,689.76,1,n
3,Acinetobacter_baumannii,60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,689.76,1,n
4,Acinetobacter_baumannii,80,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,689.76,1,n


In [86]:
avgdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1080 entries, (Acinetobacter_baumannii, 0.0) to (Streptococcus_pneumoniae, 1180.0)
Columns: 221 entries, 2R to detection
dtypes: float64(221)
memory usage: 1.8+ MB


In [117]:
# save to R
from rpy2.robjects import pandas2ri, r
import pickle
pandas2ri.activate()
rX = pandas2ri.py2ri(avgdf)
r.assign('AVG', rX)
r("save(AVG, file='AVG.gzip', compress=TRUE)")
pandas2ri.deactivate()

In [74]:
# save to R
from rpy2.robjects import pandas2ri, r
import pickle
pandas2ri.activate()
rX = pandas2ri.py2ri(newdf)
r.assign('SPOTS', rX)
r("save(SPOTS, file='SPOTS.gzip', compress=TRUE)")
pandas2ri.deactivate()

In [95]:
from seriesmodel import SeriesModel

from featurizer import PolynomialFeaturizer

sm = SeriesModel(reference_time=9, min_time=3)
PF = PolynomialFeaturizer(n=4, reference_time=2, verbose=True)
mycoefs, myscores = PF.fit_transform(X)

Featurizing trial 0
Featurizing trial 100
Featurizing trial 200
Featurizing trial 300
Featurizing trial 400
Featurizing trial 500
Featurizing trial 600
Featurizing trial 700
Featurizing trial 800
Featurizing trial 900
Featurizing trial 1000
Regressed 1078 trials, n=4 in 86 seconds


In [16]:
X.head()

0    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: data, dtype: object

In [17]:
Z = X.to_frame()
Z['trial'] = y.index.values
Z.head()

,data,trial
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4


In [18]:
# randomly assign a trigger time
# assume it varies around 18 - 6 hours - with variance 9 - 3 hours
triggers = 18 + np.random.randn(len(Z))*5
# this would be calculated...
Z['trigger'] = triggers
Z.head()

,data,trial,trigger
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,16.076086
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,12.326643
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,11.341039
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,17.715102
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4,11.448913


## Assuming trigger is known, how to I rearrange this? ##

In [19]:
# rescale the t axis
# Z['data2'] = Z.apply(lambda x: (x['trigger'] - x['data'][:,0])/20*20, axis=1)
Z.head()

,data,trial,trigger
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,16.076086
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,12.326643
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,11.341039
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,17.715102
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",4,11.448913


In [20]:
def hstacker(x, trigger):
    print x.shape
    return np.hstack(((x[:,0]-trigger*np.ones(x.shape[0])).reshape(-1,1), x[:,1:])).tolist()

A = Z.iloc[0:3]
# A[['data', 'trigger']].apply(hstacker, axis=1)

#hstacker(A['data'][2], A['trigger'][2])

B = A.apply(lambda x: hstacker(x['data'], x['trigger']), axis=1)
B= B.apply(lambda x: np.array(x))
print B.head()

def stuff(x):
    print type(x)
    print x.shape
    
B.apply(lambda x: stuff(x))

(60, 220)
(60, 220)
(60, 220)
0    [[-16.0760862507, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
1    [[-12.3266425343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
2    [[-11.3410393041, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
dtype: object
<type 'numpy.ndarray'>
(60, 220)
<type 'numpy.ndarray'>
(60, 220)
<type 'numpy.ndarray'>
(60, 220)


0    None
1    None
2    None
dtype: object

In [30]:
def one_spot(x, spot):
    t = x[:,0]
    raw = x[:,spot]
    print raw.shape
    return raw

C = A['data'].apply(lambda x: one_spot(x, 2))
print C
AVG = np.array(C.iloc[0])
print AVG
for i in xrange(1, len(C)):
    AVG = np.vstack((AVG, C.iloc[i]))

print C.head()
AVG.shape
np.mean(AVG, axis=0).shape

(60,)
(60,)
(60,)
0    [0.0, -18.0, -30.0, -39.0, -55.0, -72.0, -80.0...
1    [0.0, -6.0, -2.0, -14.0, -11.0, -10.0, -16.0, ...
2    [0.0, -9.0, -8.0, -23.0, -26.0, -32.0, -28.0, ...
Name: data, dtype: object
[   0.  -18.  -30.  -39.  -55.  -72.  -80.  -85.  -86.  -98.  -92. -104.
 -101. -109. -108. -107.  -98.  -97. -112.  -99. -106. -114. -110. -106.
 -105. -105. -108. -124. -124. -123. -122. -124. -139. -139. -144. -154.
 -167. -172. -164. -166. -164. -174. -159. -165. -185. -187. -188. -184.
 -198. -193. -197. -198. -202. -203. -207. -206. -208. -210. -217. -214.]
0    [0.0, -18.0, -30.0, -39.0, -55.0, -72.0, -80.0...
1    [0.0, -6.0, -2.0, -14.0, -11.0, -10.0, -16.0, ...
2    [0.0, -9.0, -8.0, -23.0, -26.0, -32.0, -28.0, ...
Name: data, dtype: object


(60,)

In [155]:
b = [np.hstack(((x[:,0] - trigger*np.ones(Ntimes)).reshape(-1,1),x[:,1:])) for x, trigger in zip(a['data'].values, a['trigger'].values)]
print len(b)

3


In [128]:
b = np.hstack(((b[:,0]-150).reshape(-1,1), b[:,1:]))
b

array([[ -150.,     0.,     0., ...,     0.,     0.,     0.],
       [ -130.,     0.,     0., ...,     0.,     0.,     0.],
       [ -110.,     0.,     0., ...,     0.,     0.,     0.],
       ..., 
       [  990.,   -28.,  -112., ...,   -20.,   -18.,   -24.],
       [ 1010.,   -35.,  -119., ...,   -20.,   -18.,   -25.],
       [ 1030.,   -31.,  -116., ...,   -22.,   -14.,   -29.]])

In [125]:
d = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
triggers[0]

19.688071950091054

In [122]:
print d
np.hstack((d[:,1:], d[:,0].reshape(-1,1)))

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]


array([[ 2,  3,  1],
       [ 5,  6,  4],
       [ 8,  9,  7],
       [11, 12, 10]])

In [119]:
d[:,1:]

array([[ 2,  3],
       [ 5,  6],
       [ 8,  9],
       [11, 12]])

In [121]:
d[:,0].reshape(-1,1)

array([[ 1],
       [ 4],
       [ 7],
       [10]])

In [36]:
pd.read_table?

In [37]:
ecg = pd.read_table('ecg.txt', sep='\t')
ecg.head()

,Subject ID Sequence Period TRT ECG
0,1 1 0 0 0
1,1 1 1 1 0
2,2 1 0 0 0
3,2 1 1 1 0
4,3 1 0 0 0


In [38]:
with open('ecg.txt') as f:
    header = f.readline().split()
    data = f.readlines()

In [44]:
header = header[1:]

In [45]:
data = map(lambda s: s.split(), data)

AttributeError: 'list' object has no attribute 'split'

In [47]:
ECG = pd.DataFrame(data, columns=header)
print ECG.head()

  ID Sequence Period TRT ECG
0  1        1      0   0   0
1  1        1      1   1   0
2  2        1      0   0   0
3  2        1      1   1   0
4  3        1      0   0   0


In [48]:
# save to R
from rpy2.robjects import pandas2ri, r
import pickle
pandas2ri.activate()
rX = pandas2ri.py2ri(ECG)
r.assign('ECG', rX)
r("save(ECG, file='ECG.gzip', compress=TRUE)")
pandas2ri.deactivate()

In [49]:
ECG.head()

,ID,Sequence,Period,TRT,ECG
0,1,1,0,0,0
1,1,1,1,1,0
2,2,1,0,0,0
3,2,1,1,1,0
4,3,1,0,0,0


In [52]:
newdf[['trial', 'time', '9B', '36B', '46G', 'detection']].head()

,trial,time,9B,36B,46G,detection
0,0,0,0,0,0,1
1,0,20,-5,-20,-23,1
2,0,40,0,-20,-31,1
3,0,60,0,-35,-38,1
4,0,80,1,-40,-48,1


In [51]:
ECG.groupby('ID').count()

,Sequence,Period,TRT,ECG
ID,,,,
1,2,2,2,2
10,2,2,2,2
11,2,2,2,2
12,2,2,2,2
13,2,2,2,2
14,2,2,2,2
15,2,2,2,2
16,2,2,2,2
17,2,2,2,2
